In [1]:
# Dependencies
import pandas as pd
import numpy as np
import sqlite3
from pyspark import SparkContext
sc =SparkContext()
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
# Create connections to database
conn = sqlite3.connect("Data/Hotels.db")

#Load the database table into a pandas dataframe
ratings = pd.read_sql_query("select * from ratings;", conn)
conn.close()

# Preview the dataframe
ratings.head()

,index,name,reviews_date,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_userCity,reviews_userProvince
0,0,Rancho Valencia Resort Spa,2013-11-14T00:00:00Z,5.0,www.hotels.com,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,None,None
1,1,Rancho Valencia Resort Spa,2014-07-06T00:00:00Z,5.0,www.hotels.com,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,None,None
2,2,Rancho Valencia Resort Spa,2015-01-02T00:00:00Z,5.0,www.hotels.com,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,None,None
3,3,Aloft Arundel Mills,2016-05-15T00:00:00Z,2.0,www.tripadvisor.com,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA
4,4,Aloft Arundel Mills,2016-07-09T00:00:00Z,5.0,www.tripadvisor.com,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD


In [3]:
#Grouping ratings to get 5 unique ratings
rating_list = ratings['reviews_rating'].tolist()
new_list = []
for rating in rating_list:
    if rating >= 5.0: 
        new_list.append(5.0)
    elif rating >= 4 and rating < 5:
        new_list.append(4.0)
    elif rating >= 3 and rating < 4:
        new_list.append(3.0)
    elif rating >= 2 and rating < 3:
        new_list.append(2.0)
    else:
        new_list.append(1.0)        

In [4]:
# Put all letters in lower case
# Split hotel reviews_rating to "good"/"bad"
ratings["reviews_text"] = ratings["reviews_text"].str.lower()
#ratings["rating"] = np.where(ratings["reviews_rating"]>= 4, 'good', 'bad')
ratings["rating"] = new_list
ratings['reviews_text'] = ratings['reviews_text'].astype(str)
ratings.head()

,index,name,reviews_date,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,rating
0,0,Rancho Valencia Resort Spa,2013-11-14T00:00:00Z,5.0,www.hotels.com,our experience at rancho valencia was absolute...,Best romantic vacation ever!!!!,None,None,5.0
1,1,Rancho Valencia Resort Spa,2014-07-06T00:00:00Z,5.0,www.hotels.com,amazing place. everyone was extremely warm and...,Sweet sweet serenity,None,None,5.0
2,2,Rancho Valencia Resort Spa,2015-01-02T00:00:00Z,5.0,www.hotels.com,we booked a 3 night stay at rancho valencia to...,Amazing Property and Experience,None,None,5.0
3,3,Aloft Arundel Mills,2016-05-15T00:00:00Z,2.0,www.tripadvisor.com,currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,2.0
4,4,Aloft Arundel Mills,2016-07-09T00:00:00Z,5.0,www.tripadvisor.com,i live in md and the aloft is my home away fro...,ALWAYS GREAT STAY...,Laurel,MD,5.0


In [5]:
# Dependencies
import re, string

import nltk
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/janelchadiarova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/janelchadiarova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/janelchadiarova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# Remove stop words from the list
stops = stopwords.words("english")
exclude = set(string.punctuation)

In [7]:
words_list = []
preprocessed_text = []
for review in ratings["reviews_text"]:
    
    # Create a list of words per rating after the words are converted to lowercase    
    words = word_tokenize(review)
    
    # Filter to remove stop words and punctuations    
    words2 = [word for word in words if word not in stops and word not in exclude]
    
    # Add the filtered list of words
    words_list.append(words2)
    
    # Convert the list of strings back to one string
    words3 = " ".join(words2)
    
    # Add the filtered list of words
    preprocessed_text.append(words3)
   
ratings["filteredReview"] = preprocessed_text
ratings.head()

,index,name,reviews_date,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,rating,filteredReview
0,0,Rancho Valencia Resort Spa,2013-11-14T00:00:00Z,5.0,www.hotels.com,our experience at rancho valencia was absolute...,Best romantic vacation ever!!!!,None,None,5.0,experience rancho valencia absolutely perfect ...
1,1,Rancho Valencia Resort Spa,2014-07-06T00:00:00Z,5.0,www.hotels.com,amazing place. everyone was extremely warm and...,Sweet sweet serenity,None,None,5.0,amazing place everyone extremely warm welcomin...
2,2,Rancho Valencia Resort Spa,2015-01-02T00:00:00Z,5.0,www.hotels.com,we booked a 3 night stay at rancho valencia to...,Amazing Property and Experience,None,None,5.0,booked 3 night stay rancho valencia play tenni...
3,3,Aloft Arundel Mills,2016-05-15T00:00:00Z,2.0,www.tripadvisor.com,currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,2.0,currently bed writing past hr 1/2 dogs barking...
4,4,Aloft Arundel Mills,2016-07-09T00:00:00Z,5.0,www.tripadvisor.com,i live in md and the aloft is my home away fro...,ALWAYS GREAT STAY...,Laurel,MD,5.0,live md aloft home away home ... stayed 1 nigh...


In [56]:
# Remove columns that will not be used in the analysis
df = ratings.drop(columns=["reviews_text","reviews_date","reviews_sourceURLs","reviews_title","reviews_userCity","reviews_userProvince"],axis=1)
df.head()

,index,name,reviews_rating,rating,filteredReview
0,0,Rancho Valencia Resort Spa,5.0,5.0,experience rancho valencia absolutely perfect ...
1,1,Rancho Valencia Resort Spa,5.0,5.0,amazing place everyone extremely warm welcomin...
2,2,Rancho Valencia Resort Spa,5.0,5.0,booked 3 night stay rancho valencia play tenni...
3,3,Aloft Arundel Mills,2.0,2.0,currently bed writing past hr 1/2 dogs barking...
4,4,Aloft Arundel Mills,5.0,5.0,live md aloft home away home ... stayed 1 nigh...


In [9]:
#Convert Pandas DataFrame to Spark DataFrame
spark_ratings = sqlContext.createDataFrame(df)
spark_ratings.show(5)

+-----+--------------------+--------------+------+--------------------+
|index|                name|reviews_rating|rating|      filteredReview|
+-----+--------------------+--------------+------+--------------------+
|    0|Rancho Valencia R...|           5.0|   5.0|experience rancho...|
|    1|Rancho Valencia R...|           5.0|   5.0|amazing place eve...|
|    2|Rancho Valencia R...|           5.0|   5.0|booked 3 night st...|
|    3| Aloft Arundel Mills|           2.0|   2.0|currently bed wri...|
|    4| Aloft Arundel Mills|           5.0|   5.0|live md aloft hom...|
+-----+--------------------+--------------+------+--------------------+
only showing top 5 rows



In [10]:
# Create a length column to be used as a future feature 
from pyspark.sql.functions import length
data = spark_ratings.withColumn('length', length(spark_ratings['filteredReview']))
data.show(5)

+-----+--------------------+--------------+------+--------------------+------+
|index|                name|reviews_rating|rating|      filteredReview|length|
+-----+--------------------+--------------+------+--------------------+------+
|    0|Rancho Valencia R...|           5.0|   5.0|experience rancho...|   112|
|    1|Rancho Valencia R...|           5.0|   5.0|amazing place eve...|   202|
|    2|Rancho Valencia R...|           5.0|   5.0|booked 3 night st...|   335|
|    3| Aloft Arundel Mills|           2.0|   2.0|currently bed wri...|   125|
|    4| Aloft Arundel Mills|           5.0|   5.0|live md aloft hom...|   186|
+-----+--------------------+--------------+------+--------------------+------+
only showing top 5 rows



## Feature Transformations

In [11]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [12]:
#tokenizer = Tokenizer(inputCol="filteredReview", outputCol="token_text")
#tokenized = tokenizer.transform(spark_ratings)
#tokenized.show(3, truncate=False)

In [13]:
#stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
#filtered = stopremove.transform(tokenized)
#filtered.show(3, truncate=False)

In [14]:
#hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token',numFeatures=pow(2,4))
#hashed_df = hashingTF.transform(filtered)
#hashed_df.show(3, truncate=False)

In [15]:
#idf = IDF(inputCol="hash_token", outputCol="idf_token")
#idfModel = idf.fit(hashed_df)
#rescaledData = idfModel.transform(hashed_df)

In [16]:
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='rating',outputCol='label')
tokenizer = Tokenizer(inputCol="filteredReview", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token',numFeatures=pow(2,4))
idf = IDF(inputCol="hash_token", outputCol="idf_token")

In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['label','idf_token', 'length'], outputCol='features')

In [18]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [19]:
 # Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data)
cleaned = cleaner.transform(data)
cleaned.show(3)

+-----+--------------------+--------------+------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|index|                name|reviews_rating|rating|      filteredReview|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|
+-----+--------------------+--------------+------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|    0|Rancho Valencia R...|           5.0|   5.0|experience rancho...|   112|  0.0|[experience, ranc...|[experience, ranc...|(16,[1,4,5,6,8,9,...|(16,[1,4,5,6,8,9,...|[0.0,0.0,0.184744...|
|    1|Rancho Valencia R...|           5.0|   5.0|amazing place eve...|   202|  0.0|[amazing, place, ...|[amazing, place, ...|(16,[2,3,4,5,6,7,...|(16,[2,3,4,5,6,7,...|[0.0,0.0,0.0,0.28...|
|    2|Rancho Valencia R...|           5.0|   5.0|

In [20]:
# Show label of ham spame and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.0,0.0,0.184744...|
|  0.0|[0.0,0.0,0.0,0.28...|
|  0.0|[0.0,0.5991695591...|
|  3.0|[3.0,0.2995847795...|
|  0.0|[0.0,1.4979238978...|
|  0.0|[0.0,0.5991695591...|
|  0.0|[0.0,0.5991695591...|
|  0.0|(18,[2,3,4,5,10,1...|
|  0.0|[0.0,0.5991695591...|
|  0.0|[0.0,0.5991695591...|
|  0.0|[0.0,0.2995847795...|
|  0.0|[0.0,0.2995847795...|
|  0.0|[0.0,1.4979238978...|
|  0.0|[0.0,0.2995847795...|
|  0.0|[0.0,0.8987543387...|
|  1.0|[1.0,0.5991695591...|
|  0.0|[0.0,0.5991695591...|
|  2.0|[2.0,0.5991695591...|
|  0.0|[0.0,0.5991695591...|
|  1.0|[1.0,0.0,0.738976...|
+-----+--------------------+
only showing top 20 rows



In [21]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

In [22]:
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [23]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(3)

+-----+--------------------+--------------+------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|index|                name|reviews_rating|rating|      filteredReview|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------+------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1008|Hotel Zetta San F...|           4.8|   4.0|bad expensive add...|    74|  1.0|[bad, expensive, ...|[bad, expensive, ...|(16,[1,3,6,8,9,12...|(16,[1,3,6,8,9,12...|(18,[0,2,4,7,9,10...|[-35.922869194047...|[3.09163122726791...|       1.0|
| 1010|Hotel Zetta San F...|

In [24]:
# rating: 5 - 0.0, 4 - 1.0, 3 - 2.0, 2 - 3.0, 1 - 4.0
test_results.select(['label', 'prediction']).show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  3.0|       4.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 10 rows



In [25]:
 # Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting ratings was: ", acc)

Accuracy of model at predicting ratings was:  0.7747829328081407


## Word Importance 

In [26]:
import math
from textblob import TextBlob as tb

In [27]:
def tf(word, review):
    return review.words.count(word) / len(review.words)

def n_containing(word, reviewlist):
    return sum(1 for review in reviewlist if review in review.words)

def idf(word, reviewlist):
    return math.log(len(reviewlist) / (1 + n_containing(word, reviewlist)))

def tfidf(word, review, reviewlist):
    return tf(word, review) * idf(word, reviewlist)

In [57]:
df.dtypes

index              object
name               object
reviews_rating    float64
rating            float64
filteredReview     object
dtype: object

In [58]:
good_reviews = df[(df['rating'] >= 5.0)]
good_reviews.head()

,index,name,reviews_rating,rating,filteredReview
0,0,Rancho Valencia Resort Spa,5.0,5.0,experience rancho valencia absolutely perfect ...
1,1,Rancho Valencia Resort Spa,5.0,5.0,amazing place everyone extremely warm welcomin...
2,2,Rancho Valencia Resort Spa,5.0,5.0,booked 3 night stay rancho valencia play tenni...
4,4,Aloft Arundel Mills,5.0,5.0,live md aloft home away home ... stayed 1 nigh...
5,5,Aloft Arundel Mills,5.0,5.0,stayed family daughters wedding accommodating ...


In [59]:
bad_reviews = df.loc[(df['rating'] <= 2.0)]
bad_reviews

,index,name,reviews_rating,rating,filteredReview
3,3,Aloft Arundel Mills,2.00,2.0,currently bed writing past hr 1/2 dogs barking...
26,26,EconoLodge,1.00,1.0,getting bait switch decided 'd rather stay any...
27,27,EconoLodge,1.00,1.0,choice stay tornado hit area vineland without ...
30,30,Hampton Inn Suites National HarborAlexandria Area,2.00,2.0,heat room work properly tv remote broken exces...
102,102,The Boxer,2.50,2.0,staff n't helpful one tried come clean room se...
108,108,The Boxer,2.90,2.0,breakfast 17 bucks bit disgrace particular bui...
109,109,The Boxer,2.90,2.0,bad booked 2 rooms 3 night boston trip king ro...
140,140,The Boxer,2.50,2.0,bad restaurant closed refurb result eat breakf...
142,142,The Boxer,2.90,2.0,bad want say paying 330 one night would nice g...
155,155,Fremont Hotel & Casino,2.10,2.0,bad thumping music night long loud windows vib...


In [60]:
# Create the reviewlist from df["lines"]
good_reviewlist = [tb(review) for review in good_reviews["filteredReview"]]
good_reviewlist

[TextBlob("experience rancho valencia absolutely perfect beginning end felt special happy stayed would come back heart beat"),
 TextBlob("amazing place everyone extremely warm welcoming 've stayed top notch places definitely top 2. great romantic getaway take kids along couple stuffed animals waiting girls upon arrival ca n't wait go back"),
 TextBlob("booked 3 night stay rancho valencia play tennis since one highest rated tennis resorts america place really top luxury standpoint overall experience villas really perfect staff great attention details includes fresh squeezed orange juice morning restaurants bar room service amazing tennis program really impressive well want come back"),
 TextBlob("live md aloft home away home ... stayed 1 night 7-7-16 ... staff great especially olivia extra special remembered voice phone ... tells alert pays attention customer needs.and thumbs ..."),
 TextBlob("stayed family daughters wedding accommodating staff olivia excellent rooms well maintained wou

In [62]:
# Create an empty list to be filled with text blobs from cleaning poemlist
good_reviewlist2 = []

# Loop through the poemlist
for i in range(0, len(good_reviewlist)):
    
    # Remove words that are shorter than 2 characters
    new_string = ' '.join([w for w in str(good_reviewlist[i]).split() if len(w) > 2])
    
    # Replace emm dash with space
    new_string2 = new_string.replace("—", " ")
    
    # Convert string to text blob
    new_string2 = tb(new_string2)

    # Append the text blob to the list of text blobs
    good_reviewlist2.append(new_string2)
    
good_reviewlist2

[TextBlob("experience rancho valencia absolutely perfect beginning end felt special happy stayed would come back heart beat"),
 TextBlob("amazing place everyone extremely warm welcoming 've stayed top notch places definitely top great romantic getaway take kids along couple stuffed animals waiting girls upon arrival n't wait back"),
 TextBlob("booked night stay rancho valencia play tennis since one highest rated tennis resorts america place really top luxury standpoint overall experience villas really perfect staff great attention details includes fresh squeezed orange juice morning restaurants bar room service amazing tennis program really impressive well want come back"),
 TextBlob("live aloft home away home ... stayed night 7-7-16 ... staff great especially olivia extra special remembered voice phone ... tells alert pays attention customer needs.and thumbs ..."),
 TextBlob("stayed family daughters wedding accommodating staff olivia excellent rooms well maintained would highly recomm

In [ ]:
# Calculate the most important words
#impt_words = []
#for i, review in enumerate(good_reviewlist2):
#    scores = {word: tfidf(word, review, good_reviewlist2) for word in review.words}
#    sorted_words = sorted(scores.items(), key = lambda x: x[1], reverse = True)
    
#    for word, score in sorted_words[:5]:
#       impt_words.append((i + 1, word, round(score, 5)))

In [32]:
# Create a dataframe of important words per review
#df2 = pd.DataFrame(impt_words, columns = ["review_number", "word", "TD-IDF"])

#df2["rating"] = new_list

# Remove 
#df2.head()